In [15]:
import os
import json
import requests
import time

from utils import DBHelper
from pydantic import BaseModel

ModuleNotFoundError: No module named 'pydantic'

In [16]:
db = DBHelper('./.mylogin.cnf')   # .mylogin.cnf의 경로를 인자로 줘야 합니다. 

In [17]:
print(db.query('select * from problem where name is not null'))

({'id': 1000, 'tier': 1, 'exp': 480, 'name': 'A+B'}, {'id': 1001, 'tier': 1, 'exp': 480, 'name': 'A-B'}, {'id': 1002, 'tier': 7, 'exp': 4276, 'name': '??'}, {'id': 1003, 'tier': 8, 'exp': 6329, 'name': '???? ??'}, {'id': 1004, 'tier': 8, 'exp': 6329, 'name': '?? ??'}, {'id': 1005, 'tier': 13, 'exp': 48699, 'name': 'ACM Craft'}, {'id': 1006, 'tier': 18, 'exp': 408280, 'name': '??? ???'}, {'id': 1007, 'tier': 14, 'exp': 74023, 'name': '?? ??'}, {'id': 1008, 'tier': 2, 'exp': 672, 'name': 'A/B'}, {'id': 1009, 'tier': 3, 'exp': 954, 'name': '????'}, {'id': 1010, 'tier': 6, 'exp': 2909, 'name': '?? ??'}, {'id': 1011, 'tier': 11, 'exp': 21288, 'name': 'Fly me to the Alpha Centauri'}, {'id': 1012, 'tier': 9, 'exp': 9430, 'name': '??? ??'}, {'id': 1013, 'tier': 11, 'exp': 21288, 'name': 'Contact'}, {'id': 1014, 'tier': 17, 'exp': 265117, 'name': '??'}, {'id': 1015, 'tier': 7, 'exp': 4276, 'name': '?? ??'}, {'id': 1016, 'tier': 15, 'exp': 112885, 'name': '?? ?? ?'}, {'id': 1017, 'tier': 18, 'ex

In [19]:
boj_url = "https://www.acmicpc.net"
solved_url = "https://solved.ac"
db = DBHelper('.mylogin.cnf')
tier_to_exp = dict()
tag_id_to_name = dict()
tag_name_to_id = dict()

def initialize():
    res = db.query('select * from tier_exp')
    for row in res:
        tier_to_exp[row['tier']] = row['exp']
    res = db.query('select * from tag')
    for row in res:
        tag_id_to_name[row['id']] = row['name']
        tag_name_to_id[row['name']] = row['id']
    print(tag_id_to_name)


def get_last_problem():
    url = "https://www.acmicpc.net/problemset?sort=no_desc"
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    last_num = int(soup.find('table', id='problemset').find('tbody').find('tr').find('td').text)
    return last_num

initialize()

{111: '0_1_bfs', 87: '2_sat', 175: 'a_star', 23: 'ad_hoc', 113: 'aho_corasick', 135: 'alien', 45: 'arbitrary_precision', 17: 'arithmetic', 86: 'articulation', 29: 'backtracking', 169: 'bayes', 115: 'bellman_ford', 109: 'berlekamp_massey', 16: 'bfs', 107: 'biconnected_component', 158: 'bidirectional_search', 15: 'binary_search', 183: 'bipartite_graph', 49: 'bipartite_matching', 24: 'bitmask', 93: 'bitset', 9: 'bruteforcing', 145: 'burnside', 96: 'cactus', 75: 'calculus', 26: 'case_work', 83: 'centroid', 84: 'centroid_decomposition', 69: 'cht', 20: 'combinatorics', 22: 'constructive', 59: 'convex_hull', 60: 'coordinate_compression', 99: 'crt', 172: 'dancing_links', 5: 'data_structures', 177: 'delaunay', 71: 'deque', 19: 'dfs', 179: 'differential_cryptanalysis', 25: 'dijkstra', 165: 'directed_mst', 178: 'discrete_kth_root', 146: 'discrete_log', 161: 'discrete_sqrt', 28: 'disjoint_set', 27: 'divide_and_conquer', 100: 'divide_and_conquer_optimization', 153: 'dominator_tree', 3: 'dp', 42: 'd

In [10]:
def update_problems():
    # can get problem by id array with query param at most 100

    # 1. fetch all problems and tags from db
    problems = db.query('select * from problem')
    db_problem_list = dict()
    db_problem_tags = dict()

    for p in problems:
        db_problem_list[p['id']] = Problem(
            id=p['id'],
            tier=p['tier'],
            name=p['name'],
            tag=list()
        )
        db_problem_tags[p['id']] = set()

    for pair in db.query('select * from problem_tag'):
        pid = pair['problem_id']
        tag_id = pair['tag_id']
        tag_name = tag_id_to_name[tag_id]
        db_problem_tags[pid].add(tag_name)

    db_all_tag = set()
    for tag in db.query('select * from tag'):
        db_all_tag.add(tag['name'])

    # 2. fetch problems from solved.ac
    start_num = 1000
    last_num = 1100
    lookup_url = solved_url + "/api/v3/problem/lookup"
    batch_size = 100

    for batch_start in range(start_num, last_num+1, batch_size):
        id_list = ",".join([str(i) for i in range(batch_start, batch_start+batch_size)])
        payload = {"problemIds":id_list}
        r = requests.get(lookup_url, params=payload)

        for problem in r.json():
            problem_id = problem['problemId']
            name = problem['titleKo']
            tier = problem['level']
            exp = tier_to_exp[exp]
            tags = set(d['key'] for d in problem['tags'])

            # check if problem update is necessary
            if problem_id not in db_problem_list:
                # new problem
                db.query(f'INSERT INTO problem (id, tier, exp, name) \
                        VALUES ({problem_id}, {tier}, {exp}, \'{name}\')')

            elif db_problem_list[problem_id]['tier'] != tier or db_problem_list[problem_id]['name'] != name:
                # update problem
                db.query(f'UPDATE problem SET tier={tier}, exp={exp}, name=\'{name}\' WHERE id={id}')

            # check if there's new tag
            for tag_name in tags:
                if tag not in db_all_tag:
                    tag_id = len(db_all_tag) + 1
                    db.query(f'INSERT INTO tag(id, name) VALUES({tag_id}, \'{tag_name}\')')
                    tag_name_to_id[tag_name] = tag_id
                    tag_id_to_name[tag_id] = tag_name

            # check if tag set is different
            db_tag_set = db_problem_tags.get(problem_id, set())
            if tags != db_tag_set:
                to_be_inserted = tags - db_tag_set
                to_be_deleted = db_tag_set - tags

                for tag_name in to_be_inserted:
                    tag_id = tag_name_to_id[tag_name]
                    db.query(f'INSERT INTO problem_tag(problem_id, tag_id) VALUES({problem_id, tag_id})')

                for tag_name in to_be_deleted:
                    tag_id = tag_name_to_id[tag_name]
                    db.query(f'DELETE FROM problem_tag WHERE problem_id={problem_id} AND tag_id={tag_id}')

In [11]:
update_problems()

NameError: name 'Problem' is not defined

In [8]:
query_string = f' \
            SELECT * \
            FROM tag \
            ORDER BY RAND() \
            LIMIT 6'
tags = db.query(query_string)

problems = dict()
for tag in tags:
    tag_id = tag['id']
    tag_name = tag['name']
    query_string = f' \
                SELECT problem.id, problem.tier, problem.exp \
                FROM problem join problem_tag on problem.id = problem_tag.problem_id \
                WHERE problem_tag.tag_id = {tag_id} \
                LIMIT 3'
    print(db.query(query_string))

({'id': 1077, 'tier': 20, 'exp': 977727}, {'id': 1151, 'tier': 17, 'exp': 265117}, {'id': 1218, 'tier': 25, 'exp': 9178407})
({'id': 1209, 'tier': 16, 'exp': 172714}, {'id': 3685, 'tier': 24, 'exp': 5827560}, {'id': 12010, 'tier': 25, 'exp': 9178407})
({'id': 1180, 'tier': 23, 'exp': 3711822}, {'id': 2111, 'tier': 20, 'exp': 977727}, {'id': 3748, 'tier': 20, 'exp': 977727})
({'id': 1097, 'tier': 16, 'exp': 172714}, {'id': 1305, 'tier': 17, 'exp': 265117}, {'id': 1498, 'tier': 17, 'exp': 265117})
({'id': 7610, 'tier': 25, 'exp': 9178407}, {'id': 10724, 'tier': 26, 'exp': 14501883}, {'id': 13539, 'tier': 25, 'exp': 9178407})
({'id': 1170, 'tier': 18, 'exp': 408280}, {'id': 1180, 'tier': 23, 'exp': 3711822}, {'id': 2111, 'tier': 20, 'exp': 977727})


In [7]:
print(db.topk_problems('seastar105', 10))

[{'id': 14640, 'tier': 29, 'exp': 58292339}, {'id': 18236, 'tier': 28, 'exp': 36546921}, {'id': 18237, 'tier': 28, 'exp': 36546921}, {'id': 14347, 'tier': 27, 'exp': 22985485}, {'id': 11714, 'tier': 27, 'exp': 22985485}, {'id': 17646, 'tier': 27, 'exp': 22985485}, {'id': 18527, 'tier': 27, 'exp': 22985485}, {'id': 18465, 'tier': 26, 'exp': 14501883}, {'id': 14346, 'tier': 26, 'exp': 14501883}, {'id': 10350, 'tier': 25, 'exp': 9178407}]


In [6]:
# db.query(query_string) : query_string의 결과를 보여줍니다.
# 쿼리 결과에서 하나의 row를 dictionary로서 가지는 tuple을 반환합니다.
print(db.query('SELECT * FROM problem LIMIT 5'))
# 쿼리문이 잘못 됐을 경우에는 None을 return하고 이를 알려줍니다.
print(db.query('WRONG QUERY'))

({'id': 1000, 'tier': 1, 'exp': 480}, {'id': 1001, 'tier': 1, 'exp': 480}, {'id': 1002, 'tier': 7, 'exp': 4276}, {'id': 1003, 'tier': 8, 'exp': 6329}, {'id': 1004, 'tier': 8, 'exp': 6329})
Wrong Query String "WRONG QUERY"
None


In [4]:
# 자주 쓸만한 쿼리들은 메소드로 만들어 뒀습니다.
# db.get_problem_tags(problem_id) : 문제 번호를 사용해서 문제에 붙은 태그와 경험치를 돌려받는 함수입니다.
# DB에 없는 문제 번호일 경우 None을 반환합니다
# Unrated거나 평가할 수 없는 문제일 경우 exp가 0입니다.
res = db.get_problem_tags(14636)
print(res['exp'])
print(res['tags'])
print(db.get_problem_tags(30000)) # None

2371771
[3, 14, 27, 100]
None


In [5]:
# db.get_user_tags(handle) : 유저의 handle을 인자로 넘겨주면 해당 유저의 태그별 경험치를 반환합니다.
# DB에 존재하지 않는 유저일 경우 None을 반환합니다.
res = db.get_user_tags('seastar105')
print(res)
print(db.get_user_tags('NonExistingUser')) # None

{1: 238469433, 5: 194562445, 7: 190108987, 3: 143915043, 12: 105244529, 23: 92439689, 33: 83723330, 10: 82596719, 27: 80422969, 13: 76081999, 11: 75503834, 67: 74614208, 58: 72581875, 20: 61748287, 14: 56158047, 4: 52629146, 83: 44209423, 84: 44209423, 103: 39135255, 75: 39007734, 133: 37487368, 129: 34661896, 69: 32945960, 15: 32482173, 8: 27166052, 100: 26180146, 40: 22570527, 92: 22048625, 38: 18951319, 147: 18356814, 21: 18353669, 44: 16768378, 19: 12732468, 49: 12409809, 48: 12259687, 51: 11771500, 16: 11271497, 46: 10716662, 43: 10003123, 50: 9963884, 140: 9304640, 89: 9178407, 144: 9178407, 6: 9119815, 2: 9077072, 35: 9049275, 39: 8260515, 28: 7666139, 118: 6441779, 99: 6100647, 34: 5162670, 32: 5157517, 24: 5018594, 25: 4884644, 65: 4882551, 52: 4846566, 41: 4839265, 31: 4785029, 61: 4375240, 66: 4309488, 114: 4084240, 53: 3926111, 122: 3892137, 95: 3892137, 78: 3892137, 110: 3711822, 59: 3706778, 57: 3507965, 63: 3481157, 112: 3449012, 72: 3349498, 60: 3128653, 42: 3093820, 11